### Incident Analysis Demo

This notebook demonstrates how to use the **incident analysis functions** for railway delay investigation.

Available Functions:

 **`incident_view()`** - Returns tabular data of delays during an incident

 **`incident_view_heatmap_html()`** - Creates animated HTML heat maps with temporal progression

Requirements:

Run this notebook from anywhere within the RDM_analysis project

Ensure parquet data files are generated (run `python -m preprocessor.main`)

In [13]:
# Import functions with reload to ensure latest version
import sys
import os

# Ensure project root is in Python path
project_root = os.path.dirname(os.getcwd())  # Go up from demos to RDM_analysis
if project_root not in sys.path:
    sys.path.insert(0, project_root)

try:
    import importlib
    import outputs.utils
    importlib.reload(outputs.utils)
    from outputs.utils import incident_view, incident_view_html, incident_view_heatmap_html
    print("Successfully imported incident analysis functions")
except ImportError as e:
    print(f" Import error: {e}")
    print("Make sure you're running from the RDM_analysis directory")
except Exception as e:
    print(f" Error: {e}")

✅ Successfully imported incident analysis functions


incident_view function: outputs table

In [ ]:
# Example: Analyze incident 435059 for 30 minutes starting at 19:00
result_df, incident_start, analysis_period = incident_view(
    incident_code=434859,
    incident_date='28-APR-2024',    # When incident occurred
    analysis_date='28-APR-2024',    # Date to analyze
    analysis_hhmm='0800',           # Start time (08:00)
    period_minutes=300               # Duration to 13:00
)

print(f" Incident started: {incident_start}")
print(f" Analysis period: {analysis_period}")
print(f" Data shape: {result_df.shape}")
print(" Complete dataset:")
print(result_df)

Analyzing incident 434859 (started 28-APR-2024)
Analysis period: 28-Apr-2024 08:00 to 28-Apr-2024 13:00 (300 min)
Loading 366 station files for SU
Incident Details:
  Section Code: 04303:04730
  Incident Reason: IR
  Started: 28-APR-2024 02:00
 Incident started: 28-APR-2024 02:00
 Analysis period: 28-Apr-2024 08:00 to 28-Apr-2024 13:00 (300 min)
 Data shape: (20, 7)
 Complete dataset:
   STATION_CODE  PLANNED_CALLS  ACTUAL_CALLS  DELAYED_TRAINS_OUT  \
0         12001              3             3                   0   
1         12931             22            21                   2   
2         13702              8             8                   0   
3         15866             11            11                   0   
4         16416             26            25                   1   
5         17011              6             6                   0   
6         17132             41            41                   0   
7         18067             13            13                   0   


Network Heatmap

In [15]:
# Example: Create network heatmap for incident 434859
incident_code = 705009
incident_date = '09-AUG-2024'
analysis_date = '09-AUG-2024'
analysis_hhmm = '1900' # Start at 02:00
period_minutes = 60 # full day period
interval_minutes = 10

# Generate dynamic filename with all parameters
output_file = f'heatmap_incident_{incident_code}_{analysis_date.replace("-", "_")}_{analysis_hhmm}_period{period_minutes}min_interval{interval_minutes}min.html'

heatmap_html = incident_view_heatmap_html(
    incident_code=incident_code,
    incident_date=incident_date,
    analysis_date=analysis_date, 
    analysis_hhmm=analysis_hhmm,
    period_minutes=period_minutes,       # 1 hour total period
    interval_minutes=interval_minutes,   # Split into 10-minute intervals
    output_file=output_file
)

print(" Network heatmap created!")
print(f" File: {output_file}")
print("Open the HTML file in your browser to explore the full network heatmap")
print(" Shows vibrant continuous heatmap with clickable data for affected stations")

Loading station coordinates from: C:\Users\39342\University of Glasgow\Ji-Eun Byun - MZ-JB\MSci (Research) 2024-25\reference data\stations_ref_with_dft.json
Successfully loaded 6104 station records
Creating heatmap for incident 705009
Analysis period: 09-Aug-2024 19:00 to 09-Aug-2024 20:00
Interval size: 10 minutes
Total intervals: 6
Loading 374 station files for FR
Found coordinates for 385 stations (DFT categories A/B/C1/C2)
Found delay data for 15 affected stations
Incident section code: 53221
Incident reason: I1
Incident start time: 09-AUG-2024 18:41
Looking for STANOX codes: ['53221']
Searching for STANOX: 53221
📍 Found incident location: Hitchin (53221)
🌡️ CONTINUOUS HEATMAP CREATED! 
File: heatmap_incident_705009_09_AUG_2024_1900_period60min_interval10min.html
Time steps: 6 (10-minute intervals)
Total stations mapped: 385 (DFT categories A/B/C1/C2)
Affected stations: 15
🔥 Features: Continuous delay heatmap with fading colors + grey station dots overlay
🌐 Open the HTML file in yo